# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

In [13]:
sales.sort("grade", ascending=True).explore()

/usr/local/lib/python3.7/site-packages/turicreate/visualization/_plot.py:461: UserWarning: Displaying only the first 100 rows.
  warnings.warn("Displaying only the first {} rows.".format(maximum_rows))


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_living_sqrt,sqft_lot_sqrt,bedrooms_square,floors_square
0,3980300371,2014-09-26 00:00:00+00:00,142000.0,0.0,0.00,290.0,20875.0,1.0,0,0,1,1.0,290.0,0.0,1963.0,0.0,98024,47.530772,-121.888423,1620.0,22850.0,17.029386,144.481833,0.0,1.00
1,5702500050,2014-11-04 00:00:00+00:00,280000.0,1.0,0.00,600.0,24501.0,1.0,0,0,2,3.0,600.0,0.0,1950.0,0.0,98045,47.531643,-121.748836,990.0,22549.0,24.494897,156.527953,1.0,1.00
2,3421079032,2015-02-17 00:00:00+00:00,75000.0,1.0,0.00,670.0,43377.0,1.0,0,0,3,3.0,670.0,0.0,1966.0,0.0,98022,47.263814,-121.906416,1160.0,42882.0,25.884358,208.271457,1.0,1.00
3,2420069251,2015-02-25 00:00:00+00:00,262000.0,1.0,0.75,520.0,12981.0,1.0,0,0,5,3.0,520.0,0.0,1920.0,0.0,98022,47.208201,-121.994538,1340.0,12233.0,22.803509,113.934192,1.0,1.00
4,8658300340,2014-05-23 00:00:00+00:00,80000.0,1.0,0.75,430.0,5050.0,1.0,0,0,2,4.0,430.0,0.0,1912.0,0.0,98014,47.649943,-121.908684,1200.0,7500.0,20.736441,71.063352,1.0,1.00
5,3089000005,2014-07-24 00:00:00+00:00,150000.0,2.0,1.00,850.0,54000.0,1.5,0,0,1,4.0,850.0,0.0,1950.0,0.0,98023,47.295873,-122.377304,1550.0,14440.0,29.154759,232.379001,4.0,2.25
6,3623059027,2014-10-22 00:00:00+00:00,200000.0,2.0,0.75,780.0,55764.0,1.0,0,0,4,4.0,780.0,0.0,1945.0,0.0,98058,47.441997,-122.105084,1620.0,30847.0,27.928480,236.144024,4.0,1.00
7,6146600170,2014-07-03 00:00:00+00:00,100000.0,2.0,0.75,660.0,5240.0,1.0,0,0,4,4.0,660.0,0.0,1912.0,0.0,98032,47.388098,-122.233657,850.0,5080.0,25.690465,72.387844,4.0,1.00
8,6929602721,2015-04-08 00:00:00+00:00,95000.0,2.0,1.00,960.0,7000.0,1.0,0,0,3,4.0,960.0,0.0,1918.0,0.0,98198,47.386386,-122.307210,1850.0,8120.0,30.983867,83.666003,4.0,1.00
9,1352300580,2014-11-14 00:00:00+00:00,247000.0,1.0,1.00,460.0,4120.0,1.0,0,0,3,4.0,460.0,0.0,1937.0,0.0,98055,47.486804,-122.198990,990.0,4120.0,21.447611,64.187226,1.0,1.00


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.453059     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.591614     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [16]:
import turicreate
import math
import random
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
def print_coefficients(model):    
    # Get the degree of the polynomial
    deg = len(model.coefficients['value'])-1

    # Get learned parameters as a list
    w = list(model.coefficients['value'])

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print('Learned polynomial for degree ' + str(deg) + ':')
    w.reverse()
    print(np.poly1d(w))
    
print_coefficients(model_all)
print(all_features[14-1], all_features[13-1], all_features[5-1], all_features[4-1], all_features[3-1])

Learned polynomial for degree 17:
       14         13         5         4        3
20.02 x  + 842.1 x  + 350.1 x + 24.42 x + 8469 x + 2.749e+05
sqft_above grade sqft_living_sqrt sqft_living bathrooms


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

** sqft_above grade sqft_living_sqrt sqft_living bathrooms

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [14]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [30]:
min_rss = None
best_l1_penalty = None

for l1_penalty in np.logspace(1, 7, num=13):
    next_mse = 0.0
    # train model
    model = turicreate.linear_regression.create(training[all_features+['price']],target='price', 
                                                l1_penalty=l1_penalty,
                                                l2_penalty=0,
                                                validation_set=None,verbose=False)
    #print_coefficients(model)
    # predict on validation set 
    y_test_predicted = model.predict(validation[all_features])
    # compute squared error
    rss = ((y_test_predicted-validation['price'])**2).sum()
    print(rss, l1_penalty)
    # save squared error in list of MSE for each l2_penalty
    if min_rss is None or rss < min_rss:
        min_rss = rss
        best_l1_penalty = l1_penalty
            
print(best_l1_penalty)

625766285142460.5 10.0
625766285362394.0 31.622776601683793
625766286057885.4 100.0
625766288257224.9 316.22776601683796
625766295212186.0 1000.0
625766317206080.8 3162.2776601683795
625766386760658.0 10000.0
625766606749278.6 31622.776601683792
625767302791635.2 100000.0
625769507643886.0 316227.7660168379
625776517727024.4 1000000.0
625799062845466.8 3162277.6601683795
625883719085425.2 10000000.0
10.0


In [33]:
#training[all_features+['price']].explore()

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [ ]:
10

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

**May not be correct!**

0

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [35]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [36]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [38]:
min_rss = None
best_l1_penalty = None
l1_penalty_2_nnz = {}

for l1_penalty in l1_penalty_values:
    model = turicreate.linear_regression.create(training[all_features+['price']],target='price', 
                                                l1_penalty=l1_penalty,
                                                l2_penalty=0,
                                                validation_set=None,verbose=False)
    #print_coefficients(model)
    # predict on validation set 
    y_test_predicted = model.predict(validation[all_features])
    # compute squared error
    rss = ((y_test_predicted-validation['price'])**2).sum()
    print(rss, l1_penalty)
    # save squared error in list of MSE for each l2_penalty
    if min_rss is None or rss < min_rss:
        min_rss = rss
        best_l1_penalty = l1_penalty
    l1_penalty_2_nnz[l1_penalty] = model.coefficients['value'].nnz()
            
print(best_l1_penalty)
print(l1_penalty_2_nnz)

627492659875075.2 100000000.0
628210516770747.2 127427498.57031322
629176689541061.4 162377673.91887242
630650082718674.5 206913808.111479
632940229286984.4 263665089.87303555
636268140229529.2 335981828.6283788
641261198311187.9 428133239.8719396
648983455376337.2 545559478.1168514
660962217696068.4 695192796.1775591
677261520727858.6 885866790.4100832
701046815867076.1 1128837891.6846883
737850622829048.2 1438449888.2876658
796163109639832.8 1832980710.8324375
869018172893693.5 2335721469.0901213
966925692362084.9 2976351441.6313133
1081867592324112.2 3792690190.7322536
1244927360324958.2 4832930238.571753
1384161490235110.8 6158482110.6602545
1230794720455646.5 7847599703.514623
1229157160638600.2 10000000000.0
100000000.0
{100000000.0: 18, 127427498.57031322: 18, 162377673.91887242: 18, 206913808.111479: 18, 263665089.87303555: 17, 335981828.6283788: 17, 428133239.8719396: 17, 545559478.1168514: 17, 695192796.1775591: 17, 885866790.4100832: 16, 1128837891.6846883: 15, 1438449888.28

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [39]:
#2976351441.6313133: 10, 3792690190.7322536: 6

l1_penalty_min = 2976351441.6313133
l1_penalty_max = 3792690190.7322536

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [41]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [44]:
min_rss = None
best_l1_penalty = None
l1_penalty_2_nnz = {}

for l1_penalty in l1_penalty_values:
    model = turicreate.linear_regression.create(training[all_features+['price']],target='price', 
                                                l1_penalty=l1_penalty,
                                                l2_penalty=0,
                                                validation_set=None,verbose=False)

    # predict on validation set 
    y_test_predicted = model.predict(validation[all_features])
    # compute squared error
    rss = ((y_test_predicted-validation['price'])**2).sum()

    # save squared error in list of MSE for each l2_penalty
    if min_rss is None or rss < min_rss:
        min_rss = rss
        best_l1_penalty = l1_penalty
    
    l1_penalty_2_nnz[l1_penalty] = model.coefficients['value'].nnz()
    print(rss, l1_penalty, model.coefficients['value'].nnz())
    print_coefficients(model)
            
print(best_l1_penalty)
print(l1_penalty_2_nnz)

966925692362084.9 2976351441.6313133 10
Learned polynomial for degree 17:
      15         14        13             11        8         5
12.8 x  + 32.74 x  + 3579 x  + 1.133e+04 x  + 3666 x + 789.3 x
          4             3
 + 34.14 x + 1.796e+04 x + 2182 x + 1.961e+05
974019450084554.9 3019316638.9524155 10
Learned polynomial for degree 17:
       15         14        13        11        8         5         4
11.05 x  + 32.54 x  + 3528 x  + 9877 x  + 3358 x + 782.2 x + 34.02 x
              3
 + 1.781e+04 x + 2067 x + 1.986e+05
981188367942453.6 3062281836.2735176 10
Learned polynomial for degree 17:
       15         14        13        11        8       5        4
9.306 x  + 32.33 x  + 3478 x  + 8419 x  + 3050 x + 775 x + 33.9 x
              3
 + 1.766e+04 x + 1952 x + 2.01e+05
989328342459473.2 3105247033.5946198 10
Learned polynomial for degree 17:
       15         14        13        11        8       5         4
7.513 x  + 32.11 x  + 3421 x  + 6944 x  + 2717 x + 767 x + 33.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`? **3448968612.163437**
2. What features in this model have non-zero coefficients?

```
       14        13         5         4             3
30.01 x  + 2899 x  + 690.1 x + 32.41 x + 1.587e+04 x + 661.7 x + 2.223e+05
```

In [45]:
# NOTE: this is 6 features + intercept, not clear if intercept should be included
print(all_features[14-1], all_features[13-1], all_features[5-1], all_features[4-1], all_features[3-1], all_features[1-1])

sqft_above grade sqft_living_sqrt sqft_living bathrooms bedrooms
